In [1]:
import ast
import glob
import os
import random
import re

import numpy as np
import pandas as pd

from __future__ import annotations
from codealltag_data_processor_v2025 import CodealltagDataProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed
from pandas import DataFrame
from pandas.core.series import Series
from tqdm import tqdm
from typing import Any, Dict, Generator, List, Tuple

In [2]:
cdp_2022 = CodealltagDataProcessor(data_version='20220513', config_path=['codealltag_data_processor.yml'])

In [3]:
dataset = pd.read_csv("data_utility_dataset.csv", index_col=0)

In [4]:
dataset[dataset.OT.str.contains("http://www.uni-karlsruhe.de/~Marek.Tomczyk/", regex=True, na=False)]

,FilePath,OT,OA,OTT,OTTL,MT5O,MT5PT,MT5PA,MT5PTT,MT5PTTL
10,CodEAlltag_pXL_FINANCE/7-/77524.txt,Gibt's doch (bzw. kommt im August raus). Z.B.:...,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...",Gibt's doch ( bzw. kommt im August raus ) . Z....,Gibt's O\ndoch O\n( O\nbzw. O\nkommt O\nim O\n...,FEMALE: Isolde **Nadja**; FAMILY: Grünärml **P...,Gibt's doch (bzw. kommt im August raus). Z.B.:...,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...",Gibt's doch ( bzw. kommt im August raus ) . Z....,Gibt's O\ndoch O\n( O\nbzw. O\nkommt O\nim O\n...


In [5]:
dataset[dataset.OT.str.contains("oszedo", regex=True, na=False)]

,FilePath,OT,OA,OTT,OTTL,MT5O,MT5PT,MT5PA,MT5PTT,MT5PTTL
3574,CodEAlltag_pXL_TEENS/1-/16-/16942.txt,"<yell mode=""oszedo"">Wah!</yell>\n\n\nIhno, aka...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","<yell mode=""oszedo""> Wah ! </yell> Ihno , aka ...","<yell mode=""oszedo""> O\nWah O\n! O\n</yell> O\...",MALE: Ihno **Nils**; USER: Vzaxqjackmqw **Rdmy...,"<yell mode=""oszedo"">Wah!</yell>\n\n\nNils, aka...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","<yell mode=""oszedo""> Wah ! </yell> Nils , aka ...","<yell mode=""oszedo""> O\nWah O\n! O\n</yell> O\..."


In [6]:
dataset = dataset[~dataset.FilePath.isin(["CodEAlltag_pXL_FINANCE/7-/77524.txt", "CodEAlltag_pXL_TEENS/1-/16-/16942.txt"])]

In [7]:
len(dataset)

3739

In [8]:
dataset.reset_index(drop=True, inplace=True)

In [9]:
count = 0
fps = list()
with tqdm(total=len(dataset), smoothing=0) as progress_bar: 
    for idx, row in dataset.iterrows():
        mT5_adf = pd.DataFrame(ast.literal_eval(dataset.iloc[idx].MT5PA))
        if (~mT5_adf["Pseudonym"].isna() & (mT5_adf["Pseudonym"] != "")).all():
            count += 1
            fps.append(row.FilePath)
        progress_bar.update(1)

print(f'Total filtered out: {len(dataset)-count}')

100%|█████████████████████████████████████| 3739/3739 [00:03<00:00, 1177.95it/s]

Total filtered out: 59


In [10]:
len(fps)

3680

In [11]:
selected_3500 = np.random.RandomState(
    cdp_2022.get_random_seed()
).choice(
    fps, size=3500, replace=False
).tolist()

In [12]:
len(selected_3500)

3500

In [13]:
dataset_3500 = dataset[dataset.FilePath.isin(selected_3500)]

In [14]:
len(dataset_3500)

3500

In [15]:
annotation_df = cdp_2022.get_annotation_df()

In [16]:
filtered_adf = annotation_df[
    annotation_df.FilePath.isin(
        dataset_3500.FilePath.str.replace('.txt', '.ann', regex=True)
    )
]

In [17]:
cdp_2022.get_category_or_label_wise_count_or_ratio(filtered_adf, category_wise=False)

{'UFID': 248,
 'USER': 250,
 'ORG': 452,
 'DATE': 562,
 'STREETNO': 757,
 'ZIP': 767,
 'EMAIL': 771,
 'STREET': 810,
 'PHONE': 875,
 'CITY': 1465,
 'URL': 1640,
 'FEMALE': 2240,
 'FAMILY': 2391,
 'MALE': 3008}

In [18]:
dataset_3500.reset_index(drop=True, inplace=True)
dataset_3500.to_csv("data_utility_dataset_3500.csv")